In [42]:
import functools
import glob
import math
import json
import os
import time

from os import path

import ee
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from multiprocessing import Pool
from geographiclib.geodesic import Geodesic
from geopy.geocoders import Nominatim
from tqdm import notebook as tqdm
from collections import Counter
import rasterio
from shapely.geometry import Point, Polygon

from typing import List, Tuple
import geopandas as gpd

# Modules

In [43]:
def sample_points_in_polygon(polygon: Polygon,
                             num_samples: float = 1,
                             add_padding=False) -> List[Tuple]:
    """Sample point(s) within a given polygon.

    This function `num_samples` samples points that lies within the bounds of a
    given polygon. First, a point is uniformly sampled that are within the
    minimum and maximum latitude and longitude of the polygon. Then the point
    is checked whether it is within the polygon. These two steps are repeated
    until there are `num_samples` points that are returned.

    Args:
        polygon (Polygon): A polygon that containts all points that are
            randomly sampled.
        num_samples (float)

    Returns:
        points (List[Tuple]): A list of points. The points are representated
            as tuples of x (longitude) and y (latitude) values.
    """
    points = []
    # get boundaries (minimum and maximum latitude and longitude) of polygon
    min_x, min_y, max_x, max_y = polygon.bounds
    if add_padding:
        min_x += 1 / 180
        min_y += 1 / 180
        max_x -= 1 / 180
        max_y -= 1 / 180

    # repeat sampling process while the number of points samples is not equal
    # `num_samples`
    while len(points) < num_samples:
        # uniformly sample longitude (x) and latitude with bounds
        x, y = np.random.uniform(
            min_x, max_x), np.random.uniform(min_y, max_y)
        point = Point(x, y)
        # if point is within the polygon, add it to list `points`
        if polygon.contains(point):
            points.append((x, y))
    return points

In [44]:
def get_square_area(longitude: float, latitude: float,
                    square_length: float = 200.,
                    geod: Geodesic = None):
    """
    Args:
        lat: Latitude
        lon: Longtitude
        square_length: Measured in meters.
        geod: (Geodesic)
    Returns:
        square_pts: [lower left x, lower left y, ...
                     upper right x, upper right y]
    """
    if geod is None:
        #Define the ellipsoid
        geod = Geodesic.WGS84
    diag_len = np.sqrt(2*(square_length / 2.)**2)
    ll_point = geod.Direct(latitude, longitude, -135, diag_len)
    ur_point = geod.Direct(latitude, longitude, 45, diag_len)
    square_pts = [
        ll_point['lon2'], ll_point['lat2'],  # min x, min y
        ur_point['lon2'], ur_point['lat2']   # max x, max y
    ]
    return square_pts

In [45]:
# Function to mask clouds using the Sentinel-2 QA band
# @param {ee.Image} image Sentinel-2 image
# @return {ee.Image} cloud masked Sentinel-2 image

def mask2clouds(image):
    qa = image.select('QA60');
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and qa.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)

# Authenticate

In [46]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AdQt8qjTHpop8sYGGxdXEZx5L21uMDqua9ropbzmZoW45CyI59poLkfpRgY

Successfully saved authorization token.


# Load dataset

In [47]:
# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data.
dataset = ee.ImageCollection('COPERNICUS/S2')
dataset = dataset.filterDate('2017-01-01', '2021-07-01')
# Pre-filter to get less cloudy granules.
dataset = dataset.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) 
dataset = dataset.map(mask2clouds)
dataset = dataset.median()

In [48]:
# RGB visualization configuration
rgbVis = {
  "min": 0.0,
  "max": 0.3,
  "bands": ['B4', 'B3', 'B2']
}

# Load metadata

In [49]:
with open("../data/ground_truth/train_1200_v2.geojson") as f:
    v2 = gpd.read_file(f)
    vgdf = v2
    vname = "v2"

lon_lat_list = []

num_tr = len(vgdf[(vgdf.split == "train") & (vgdf.pos_neg == "pos")])
num_tr_low = len(vgdf[(vgdf.split == "train_lower") & (vgdf.pos_neg == "pos")])
num_tr_upp = len(vgdf[(vgdf.split == "train_upper") & (vgdf.pos_neg == "pos")])

num_va = len(vgdf[(vgdf.split.str.startswith("val") & (vgdf.pos_neg == "pos"))])
num_te_rw = len(vgdf[vgdf.split.str.startswith("test") & (vgdf.pos_neg == "pos") & (vgdf.Country == "Rwanda")])
num_te_ug = len(vgdf[vgdf.split.str.startswith("test") & (vgdf.pos_neg == "pos") & (vgdf.Country == "Uganda")])

ind = vgdf[(vgdf.split == "train-lower") & (vgdf.pos_neg == "neg")].index[0]
samples = sample_points_in_polygon(vgdf.iloc[ind].geometry, num_tr_low)
samples = [
    (s[0], s[1], vgdf.iloc[ind].id + "-{}".format(i)) 
    for i, s in enumerate(samples)]
lon_lat_list += samples

ind = vgdf[(vgdf.split == "train-upper") & (vgdf.pos_neg == "neg")].index[0]
samples = sample_points_in_polygon(vgdf.iloc[ind].geometry, num_tr_upp)
samples = [
    (s[0], s[1], vgdf.iloc[ind].id + "-{}".format(i)) 
    for i, s in enumerate(samples)]
lon_lat_list += samples

ind = vgdf[(vgdf.split == "train") & (vgdf.pos_neg == "neg")].index[0]
samples = sample_points_in_polygon(vgdf.iloc[ind].geometry, num_tr_upp)
samples = [
    (s[0], s[1], vgdf.iloc[ind].id + "-{}".format(i)) 
    for i, s in enumerate(samples)]
lon_lat_list += samples

ind = vgdf[(vgdf.split == "val") & (vgdf.pos_neg == "neg")].index[0]
samples = sample_points_in_polygon(vgdf.iloc[ind].geometry, num_va)
samples = [
    (s[0], s[1], vgdf.iloc[ind].id + "-{}".format(i)) 
    for i, s in enumerate(samples)]
lon_lat_list += samples

ind = vgdf[(vgdf.split == "test") & (vgdf.pos_neg == "neg") & (vgdf.Country == "Uganda")].index[0]
samples = sample_points_in_polygon(vgdf.iloc[ind].geometry, num_te_ug)
samples = [
    (s[0], s[1], vgdf.iloc[ind].id + "-{}".format(i)) 
    for i, s in enumerate(samples)]
lon_lat_list += samples

ind = vgdf[(vgdf.split == "test") & (vgdf.pos_neg == "neg") & (vgdf.Country == "Rwanda")].index[0]
samples = sample_points_in_polygon(vgdf.iloc[ind].geometry, num_te_rw)
samples = [
    (s[0], s[1], vgdf.iloc[ind].id + "-{}".format(i)) 
    for i, s in enumerate(samples)]
lon_lat_list += samples

lon_lat_list = sorted(lon_lat_list)

In [50]:
len(lon_lat_list)

1719

# Check data

In [30]:
folders = {
    320: "v1_b2p_rgb/v1_b2p_rgb_large_590_320",
    600: "v1_b2p_rgb/v1_b2p_rgb_large_1150_600",
    1200: "v1_b2p_rgb/v1_b2p_rgb_large_2350_1200"
}

In [42]:
v1[v1.id.str.contains("006f100000a86Fk")]

,id,GPS (Latitude),GPS (Longitude),split,min_x,max_x,Country,pos_neg,geometry
99,rw-006f100000a86Fk,-2.280762,29.206295,train_lower,28.854443,29.595857,Rwanda,pos,"POLYGON ((29.21146 -2.28596, 29.20113 -2.28596..."


In [41]:
for tile_size in [320, 600, 1200]:
    folder = folders[tile_size]
    tif_files = glob.glob(path.join(folder, "*.tif"))
    print("Tile size: {}".format(tile_size))
    print("No. of tif files: {}, no. lat lon samples {}".format(
        len(tif_files), len(lon_lat_list)))
    area_length = 2 * tile_size - 50
    for n in range(len(lon_lat_list)):
        prefix = "{}m_{}".format(area_length, lon_lat_list[n][2])
        files = glob.glob(path.join(folder, prefix) + ".tif") + glob.glob(
            path.join(folder, prefix) + "(1).tif")
        if len(files) < 0:
            print("[WARM] {} does not exist".format(prefix))
        elif len(files) > 1:
            print("[WARM] {} has more than 1 file.".format(prefix))
            print(files)
    break

Tile size: 320
No. of tif files: 3354, no. lat lon samples 3354
[WARM] 590m_train_pos_rw-006f100000a86Fx has more than 1 file.
['v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Fx.tif', 'v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Fx(1).tif']
[WARM] 590m_train_pos_rw-006f100000a86Fk has more than 1 file.
['v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Fk.tif', 'v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Fk(1).tif']
[WARM] 590m_train_pos_rw-006f100000a86Fu has more than 1 file.
['v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Fu.tif', 'v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Fu(1).tif']
[WARM] 590m_train_pos_rw-006f100000a86Ft has more than 1 file.
['v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Ft.tif', 'v1_b2p_rgb/v1_b2p_rgb_large_590_320/590m_train_pos_rw-006f100000a86Ft(1).tif']
[WARM] 590m_train_pos_rw-006f100000a86Fr has more th

In [22]:
len(unique_lonlat_names), len(unique_fnames)

(3354, 3354)

In [35]:
unique_lonlat_names.difference(unique_fnames)

{'590m_test_pos_rw-006f100000bMK0j',
 '590m_test_pos_rw-006f100000d7kjZ',
 '590m_test_pos_rw-006f100000d7bAM',
 '590m_val_pos_rw-006f100000a86Fz',
 '590m_rw-neg-te-0-133',
 '590m_test_pos_rw-006f100000d7kmY',
 '590m_rw-neg-te-0-425',
 '590m_rw-neg-te-0-341',
 '590m_test_pos_rw-006f100000bMJys',
 '590m_test_pos_rw-006f100000ZQRpz',
 '590m_rw-neg-te-0-619',
 '590m_val_pos_rw-006f100000d7EVd',
 '590m_test_pos_rw-006f100000eeqs4',
 '590m_rw-neg-te-0-503',
 '590m_val_pos_rw-006f100000a86DB',
 '590m_rw-neg-te-0-465',
 '590m_rw-neg-te-0-567',
 '590m_rw-neg-te-0-592',
 '590m_rw-neg-te-0-569',
 '590m_test_pos_rw-006f100000eekYU',
 '590m_rw-neg-te-0-316',
 '590m_rw-neg-te-0-598',
 '590m_rw-neg-te-0-46',
 '590m_test_pos_rw-006f100000d7jID',
 '590m_test_pos_rw-006f100000biKAu',
 '590m_rw-neg-te-0-129',
 '590m_rw-neg-te-0-538',
 '590m_rw-neg-te-0-210',
 '590m_val_pos_rw-006f100000d7Q9N',
 '590m_test_pos_rw-006f100000eessF',
 '590m_val_pos_rw-006f100000d6wCD',
 '590m_val_pos_rw-006f100000d7ELo',
 '5

In [27]:
len(unique_fnames.difference(unique_lonlat_names))

1242

In [36]:
unique_fnames.difference(unique_lonlat_names)

{'590m_train_pos_rw-006f100000a86Dg',
 '590m_rw-neg-tr-upper-0-575',
 '590m_train_pos_rw-006f100000eekVG',
 '590m_train_pos_rw-006f100000d761a',
 '590m_ug-neg-te-0-78',
 '590m_ug-neg-te-0-50',
 '590m_train_pos_rw-006f100000bKyu7',
 '590m_train_pos_rw-006f100000a86IW',
 '590m_rw-neg-tr-lower-0-496',
 '590m_ug-neg-te-0-119',
 '590m_train_pos_rw-006f100000a86EX',
 '590m_rw-neg-tr-lower-0-539',
 '590m_ug-neg-te-0-29',
 '590m_train_pos_rw-006f100000eeuuO',
 '590m_rw-neg-tr-upper-0-464',
 '590m_rw-neg-tr-lower-0-545',
 '590m_ug-neg-te-0-170',
 '590m_train_pos_rw-006f100000Zt6Uj',
 '590m_rw-neg-tr-lower-0-507',
 '590m_rw-neg-tr-upper-0-460',
 '590m_ug-neg-te-0-167',
 '590m_rw-neg-tr-upper-0-440',
 '590m_rw-neg-tr-upper-0-467',
 '590m_rw-neg-tr-lower-0-486',
 '590m_rw-neg-tr-upper-0-427',
 '590m_train_pos_rw-006f100000bgtSn',
 '590m_rw-neg-tr-lower-0-554',
 '590m_val_pos_rw-006f100000d7Q9m',
 '590m_train_pos_rw-006f100000d761L',
 '590m_ug-neg-te-0-132',
 '590m_ug-neg-te-0-93',
 '590m_train_pos

# Extract images from dataset

## 320m

In [51]:
i = 0

In [52]:
area_length = 2 * 320 - 50
tile_size = 320
print("Start downloading")
while i < len(lon_lat_list):
    print("i in [{};{}]".format(i, i + 100))
    for n in range(i, min(i + 100, len(lon_lat_list))):
        # desc += " {}".format(n)
        lon  = lon_lat_list[n][0]
        lat = lon_lat_list[n][1]
        geometry = ee.Geometry.Rectangle(get_square_area(lon, lat, area_length))
        visualized = dataset.clip(geometry).visualize(**rgbVis)
        fpath = "{}m_{}_{}".format(area_length, lon_lat_list[n][2], n)
        task = ee.batch.Export.image.toDrive(
            image=visualized,
            # task name to be shown in the Tasks tab
            description=fpath,
            # filename to be saved in the Google Drive
            fileNamePrefix=fpath,
            folder="{}_b2p_rgb_large_{}_{}_neg".format(
                vname, area_length, tile_size),
            scale=10, # the spatial resolution of the image
            region=geometry,
            maxPixels=1e13
        )
        task.start()
    i += 100
    print("Sleep...")
    time.sleep(10*60)

Start downloading
i in [0;100]
Sleep...
i in [100;200]
Sleep...
i in [200;300]
Sleep...
i in [300;400]
Sleep...
i in [400;500]
Sleep...
i in [500;600]
Sleep...
i in [600;700]
Sleep...
i in [700;800]
Sleep...
i in [800;900]
Sleep...
i in [900;1000]
Sleep...
i in [1000;1100]
Sleep...
i in [1100;1200]
Sleep...
i in [1200;1300]
Sleep...
i in [1300;1400]
Sleep...
i in [1400;1500]
Sleep...
i in [1500;1600]
Sleep...
i in [1600;1700]
Sleep...
i in [1700;1800]
Sleep...


## 600

In [53]:
i = 0

In [54]:
area_length = 2 * 600 - 50
tile_size = 600
print("Start downloading")
while i < len(lon_lat_list):
    print("i in [{};{}]".format(i, i + 100))
    for n in range(i, min(i + 100, len(lon_lat_list))):
        # desc += " {}".format(n)
        lon  = lon_lat_list[n][0]
        lat = lon_lat_list[n][1]
        geometry = ee.Geometry.Rectangle(get_square_area(lon, lat, area_length))
        visualized = dataset.clip(geometry).visualize(**rgbVis)
        fpath = "{}m_{}_{}".format(area_length, lon_lat_list[n][2], n)
        task = ee.batch.Export.image.toDrive(
            image=visualized,x
            # task name to be shown in the Tasks tab
            description=fpath,
            # filename to be saved in the Google Drive
            fileNamePrefix=fpath,
            folder="{}_b2p_rgb_large_{}_{}_neg".format(
                vname, area_length, tile_size),
            scale=10, # the spatial resolution of the image
            region=geometry,
            maxPixels=1e13
        )
        task.start()
    i += 100
    print("Sleep...")
    time.sleep(10*80)

Start downloading
i in [0;100]
Sleep...
i in [100;200]
Sleep...
i in [200;300]
Sleep...
i in [300;400]
Sleep...
i in [400;500]
Sleep...
i in [500;600]
Sleep...
i in [600;700]
Sleep...
i in [700;800]
Sleep...
i in [800;900]
Sleep...
i in [900;1000]
Sleep...
i in [1000;1100]
Sleep...
i in [1100;1200]
Sleep...
i in [1200;1300]
Sleep...
i in [1300;1400]
Sleep...
i in [1400;1500]
Sleep...
i in [1500;1600]
Sleep...
i in [1600;1700]
Sleep...
i in [1700;1800]
Sleep...


## 1200

In [55]:
i = 0

In [56]:
area_length = 2 * 1200 - 50
tile_size = 1200
print("Start downloading")
while i < len(lon_lat_list):
    print("i in [{};{}]".format(i, i + 100))
    for n in range(i, min(i + 100, len(lon_lat_list))):
        # desc += " {}".format(n)
        lon  = lon_lat_list[n][0]
        lat = lon_lat_list[n][1]
        geometry = ee.Geometry.Rectangle(get_square_area(lon, lat, area_length))
        visualized = dataset.clip(geometry).visualize(**rgbVis)
        fpath = "{}m_{}_{}".format(area_length, lon_lat_list[n][2], n)
        task = ee.batch.Export.image.toDrive(
            image=visualized,
            # task name to be shown in the Tasks tab
            description=fpath,
            # filename to be saved in the Google Drive
            fileNamePrefix=fpath,
            folder="{}_b2p_rgb_large_{}_{}_neg".format(
                vname, area_length, tile_size),
            scale=10, # the spatial resolution of the image
            region=geometry,
            maxPixels=1e13
        )
        task.start()
    i += 100
    print("Sleep...")
    time.sleep(10*80)

Start downloading
i in [0;100]
Sleep...
i in [100;200]
Sleep...
i in [200;300]
Sleep...
i in [300;400]
Sleep...
i in [400;500]
Sleep...
i in [500;600]
Sleep...
i in [600;700]
Sleep...
i in [700;800]
Sleep...
i in [800;900]
Sleep...
i in [900;1000]
Sleep...
i in [1000;1100]
Sleep...
i in [1100;1200]
Sleep...
i in [1200;1300]
Sleep...
i in [1300;1400]
Sleep...
i in [1400;1500]
Sleep...
i in [1500;1600]
Sleep...
i in [1600;1700]
Sleep...
i in [1700;1800]
Sleep...
